In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_addons as tfa
from numpy.random import default_rng
rng = default_rng()

In [ ]:
SEED = 42
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = PROJECT_FOLDER + "train_images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

IMAGE_SIZE = (224, 224)
VAL_SPLIT = 0.1

In [ ]:
train_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='training',
    validation_split=VAL_SPLIT
)

val_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='validation',
    validation_split=VAL_SPLIT
)

test_ds = image_dataset_from_directory(
    directory=TEST_DATA_FOLDER,
    labels=None,
    batch_size=1,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
)

In [ ]:
n_classes = len(train_ds.class_names)
n_classes

In [ ]:
def random_cutout(images_tensor, labels):
    ratios_x = rng.uniform(0.2, 0.6)*images_tensor.shape[1]
    ratios_y = rng.uniform(0.2, 0.6)*images_tensor.shape[2]

    xs_mask = tf.cast(ratios_x, tf.int32)
    xs_mask = xs_mask if xs_mask % 2 == 0 else xs_mask+1 # force number even

    ys_mask = tf.cast(ratios_y, tf.int32)
    ys_mask = ys_mask if ys_mask % 2 == 0 else ys_mask+1 # force number even

    return tfa.image.random_cutout(images_tensor, (xs_mask, ys_mask), constant_values = (255.0, 0.0, 0.0)), labels

train_ds = train_ds.map(random_cutout)
val_ds = val_ds.map(random_cutout)

# Models

In [ ]:
def create_baseline_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top=True,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    return keras.Sequential([
        efficientnet_layer,
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes, activation='softmax')
    ])


In [ ]:
model = create_baseline_efficientnet_model(n_classes)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckp',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
]

history = model.fit(train_ds, epochs=25, callbacks=callbacks, validation_data=val_ds)

In [ ]:
model.save('empanadanet.h5')